## Data Ingestion Pipeline

## Document Parsing

In [ ]:
import sys
!{sys.executable} -m pip install langchain langchain-core langchain-community pypdf pymupdf sentence-transformers faiss-cpu chromadb langchain-groq python-dotenv typesense langchain_openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.9 MB/s eta

In [ ]:
# Document Structure
from langchain_core.documents import Document

In [ ]:
doc = Document(
    page_content="Machine learning is fun",
    metadata = {                                                      # to apply filters
        "source" : "example.txt",
        "page_no" : 1,
        "author" : "firdous",
        "date_created" : "2025-01-01",
    }
)
doc

Document(metadata={'source': 'example.txt', 'page_no': 1, 'author': 'firdous', 'date_created': '2025-01-01'}, page_content='Machine learning is fun')

In [ ]:
# create simple txt file
import os
os.makedirs('data/text_files',exist_ok=True)

In [ ]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",

    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


    """
}

for filepath,content in sample_texts.items():
  with open(filepath,'w',encoding="utf-8") as f:
    f.write(content)
print("Sample text files created")

Sample text files created


In [ ]:
# TextLoader
from langchain_community.document_loaders import TextLoader
loader = TextLoader('data/text_files/python_intro.txt')
document = loader.load()
print(document)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [ ]:
# DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    'data/text_files',
    glob = "**/*.txt",
    loader_cls = TextLoader,
    loader_kwargs={'encoding' : 'utf-8'},
    show_progress = True
)

documents = dir_loader.load()
documents

100%|██████████| 2/2 [00:00<00:00, 2031.14it/s]


[Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '),
 Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogram

In [ ]:
#
from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader #PyMuPDF better to PyPDF

dir_loader = DirectoryLoader(
    'data/pdf_files',
    glob = "**/*.pdf",
    loader_cls = PyMuPDFLoader,
    show_progress=True
)

documents = dir_loader.load()
documents
# Visit langchain documentation for other types of documents loader



  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.70it/s]


[Document(metadata={'producer': '', 'creator': '', 'creationdate': '', 'source': 'data/pdf_files/1.ICASERT_2019_paper_690.pdf', 'file_path': 'data/pdf_files/1.ICASERT_2019_paper_690.pdf', 'total_pages': 7, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/337768246\nBrain Tumor Detection Using Convolutional Neural Network\nConference Paper · December 2019\nDOI: 10.1109/ICASERT.2019.8934561\nCITATIONS\n338\nREADS\n21,110\n5 authors, including:\nTonmoy Hossain\nUniversity of Virginia\n31 PUBLICATIONS\xa0\xa0\xa0702 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nFairuz Shadmani Shishir\nUniversity of Kansas\n14 PUBLICATIONS\xa0\xa0\xa0480 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nMohsena Ashraf\nAhsanullah University of Science and Technology\n13 PUBLICATIONS\xa0\xa0\xa0515 CITAT

## RAG Pipeline : Data Ingestion to vector DB pipeline

In [ ]:
import os
from langchain_community.document_loaders import  PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
# Read all the pdfs inside directory
def process_all_pdfs(pdf_directory):
  """Process all pdfs in a directory"""

  all_documents = []
  pdf_dir = Path(pdf_directory)

  pdf_files = list(pdf_dir.glob("**/*.pdf"))
  print(f"Found {len(pdf_files)} PDF files to process")

  for pdf_file in pdf_files:
    print(f'\nProcessing : {pdf_file.name}')
    try:
      loader = PyMuPDFLoader(str(pdf_file))
      documents = loader.load()

      for doc in documents:
        doc.metadata['source_file'] = pdf_file.name
        doc.metadata['file_type'] = 'pdf'

      all_documents.extend(documents)
      print(f"Loaded {len(documents)} pages")
    except Exception as e:
      print(f"Error : {e}")

  print(f"Total documents loaded: {len(all_documents)}")
  return all_documents


In [ ]:
all_pdf_documents = process_all_pdfs('data/pdf_files')

Found 3 PDF files to process

Processing : Firdous Shaikh.pdf
Loaded 1 pages

Processing : Placement brochure.pdf
Loaded 24 pages

Processing : 1.ICASERT_2019_paper_690.pdf
Loaded 7 pages
Total documents loaded: 32


In [ ]:
# Text Splitting get into chuncks
def split_documents()